# <center>Projeto ATD</center>
## <center>2ºsemestre 2020/2021</center>
### <center>FCTUC - DEI</center>
### Membros:
* Joel Oliveira - 2019227468
* Tomás Mendes - 2019232272


#### Livrarias utilizadas neste projeto:
* numpy
* pandas
* matplotlib
* scipy
* sklearn
* seaborn

#### Todas estas livrarias podem ser instaladas com recurso ao pip, na linha de comandos, ou ao conda, na consola do Conda.


In [ ]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import scipy as sy
from numpy.fft import fft, fftshift
import seaborn as sns
import matplotlib
from sklearn.metrics import confusion_matrix
from IPython.display import Markdown as md

from data import *
from functions import *

#Colocar a False para gráficos claros (concordância com theme branco do jupyter)
sns.set_style("whitegrid")
sns.set_palette("deep")

#### Leitura do ficheiro 'labels.txt'
#### A Leitura do ficheiro 'activity_labels.txt' é realizada no ficheiro 'data.py', e os dados são importados desse ficheiro

In [ ]:
# Filtrar dados do ficheiro para serem recebidos dados dos 8 primeiros voluntários
intervalos = np.array(list(filter(lambda x: x[0]<=8, np.loadtxt('labels.txt', dtype = int))))

## Verificação dos dados presentes no DataSet, através do método 'info()' da <i>DataFrame</i> do módulo <i>Pandas</i>. 

Esta função devolve informações sobre a informação presente na <i>DataFrame</i>. É devolvida uma <i>string</i> com:
* quantidade de valores nulos e não nulos
* tipo de dados em cada coluna

Assim, é possível verificar que o <i>DataSet</i> está bastante completo, sem falhas de informação e o tipo de dados presentes, do tipo <i>Float</i> neste caso.


In [ ]:
for i in range(len(users)):
    print("//" + "_"*15 + f"Sujeito {i//2+1} Experiencia {i%2+1}" + "_"*15 + "\\\\", end='\n\n')
    print(users[i].info())
    print()

## Atividades presentes nos <i>Datasets<i>
### Atividades Dinâmicas:
* WALKING
* WALKING_UPSTAIRS
* WALKING_DOWNSTAIRS

### Atividades Estáticas:
* SITTING
* STANDING
* LAYING

### Atividades de Transição:
* STAND_TO_SIT
* SIT_TO_STAND
* SIT_TO_LIE
* LIE_TO_SIT
* STAND_TO_LIE
* LIE_TO_STAND

## 2. Visualização e identificação das atividades do sinal de cada experiência

In [ ]:
user = get_user(1,1)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(1,2)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(2,3)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(2,4)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(3,5)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(3,6)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(4,7)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

In [ ]:
user = get_user(4,8)
coordenadas = np.array(list(map(lambda y: [y[-3], y[-2]], filter(lambda x: x[0] == 1 and x[1] == 1, intervalos))))
upcoords = list(map(lambda x: x[1], coordenadas[1::2]))
downcoords = list(map(lambda x: x[1], coordenadas[::2]))

uplabels = list(map(lambda x: x[0], coordenadas[1::2]))
downlabels = list(map(lambda x: x[0], coordenadas[::2]))

plot_with_labels(upcoords, downcoords, uplabels, downlabels, user, 0)

## 3. Identificação das caraterísticas principais das várias atividades

$
\text{Para Calcular a amplitude Cm: } \\
X_{DFT}[m] - \text{magnitude dos pontos na DFT} \\
c_{m} = \frac{X_{DFT}[m]}{N} -> N = len(\text{ufrag}) \\
C_{m} = \begin{cases} \frac{|X_{DFT}[0]|}{N}, \mbox{ if } m \mbox{ = 0} \\ 
\frac{2|X_{DFT}[m]|}{N}, \mbox{ if } m\mbox{ > 0} \end{cases}
$

## Seleção da janela para segmentar as atividades
Nos <i>slides</i> das <i>TP's<i> há 3 exemplos de janelas usadas para estudos de frequências associadas a sinais. Estas são a janela de <i>Blackman</i>,de <i>Hamming</i> e de <i>Hanning</i>. Na documentação do módulo <i>numpy</i>, é possível encontrar mais algumas com história. A janela de <i>Bartlett</i>, que tem a forma de um triângulo, e a janela de <i>Kaiser</i>, que tem como parâmetro um $\beta$ que na prática serve como fator de equilibrio tempo-frequência na análise do sinal.
    
Para o caso vamos testar apenas as 3 primeiras referidas.

In [ ]:
uteste = walks_user1_1[0]
size = len(uteste)
bwindow = signal.windows.blackman(size)
hmwindow = np.hamming(size)
hnwindow = signal.windows.hann(size)
beta = 7.5
kwindow = signal.windows.kaiser(size, beta)
## PLOTTING 
fig = plt.figure(figsize=(14,5))

plt.plot(bwindow, label = 'Blackman')
plt.plot(hmwindow, label = 'Hamming')
plt.plot(hnwindow, label = 'Hann')

plt.title("Windows Distribution")
plt.tick_params(axis='x')
plt.tick_params(axis='y')

plt.legend()
plt.tight_layout()

Pela Documentação do <i>Numpy</i> foi implementado código para verificação visual de resposta a frequência de cada uma destas janelas 

In [ ]:
fbwindow = np.abs(fftshift(fft(bwindow,1024)))
fhmwindow = np.abs(fftshift(fft(hmwindow,1024)))
fhnwindow = np.abs(fftshift(fft(hnwindow,1024)))

freq = np.linspace(-0.5, 0.5, 1024)

with np.errstate(divide='ignore', invalid='ignore'):
    nfbwindow = 20*np.log10(fbwindow)
    nfhmwindow = 20* np.log10(fhmwindow)
    nfhnwindow = 20 * np.log10(fhnwindow)

nfbwindow = np.clip(nfbwindow, -150, 100)
nfhmwindow = np.clip(nfhmwindow, -150, 100)
nfhnwindow = np.clip(nfhnwindow, -150, 100)
    
fig = plt.figure(figsize=(14,5))

plt.plot(freq, nfbwindow, label = 'Blackman')
plt.plot(freq, nfhmwindow, label = 'Hamming')
plt.plot(freq, nfhnwindow, label = 'Hann')

plt.title("Windows Distribution")
plt.tick_params(axis='x')
plt.tick_params(axis='y')


plt.legend()
plt.tight_layout()

Pelos gráficos das janelas no tempo e da resposta a frequência das janelas a pensamos que a mais apropriada para o este projeto será a <i>Hanning</i>, apesar de que na prática e com testes feitos anteriormente, os resultados serem basicamente os mesmos. A janela de <i>Hanning</i>, pelo nossa pesquisa, tem uma eficiência, na computação, similar à da janela de <i>Hamming</i>. Obtém resultados piores no domínio do tempo, pois a sua janela atinge 0 nas pontas, o que pode alterar bastante o sinal original. 
No entanto, para a análise das frequências, isto pode não ser algo negativo. Isto faz com que tenhamos a certeza de que não é adicionada nenhuma frequência indesejada na transformada. Pelos fragmentos feitos das atividades, também nos parece que com a quantidade de dados em cada fragmento, o abate das pontas não seja significativo para a perda total das frequências fundamentais à análise da DFT. Para além disto, no caso da STFT, será efetuada uma sobreposição de janelas, pelo que a maioria dessas frequências abatidas, será recuperada na janela seguinte.

In [ ]:
fig = plt.figure(figsize=(8,5))
accax = 'Y'
plt.subplot(311)
plt.plot(bwindow*uteste[accax])
plt.title("Blackman Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')



plt.subplot(312)
plt.plot(hmwindow*uteste[accax])
plt.title("Hamming Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')


plt.subplot(313)
plt.plot(hnwindow*uteste[accax])
plt.title("Hanning Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')


plt.tight_layout()

In [ ]:
uteste = walks_user1_1_detrended[0]

fig = plt.figure(figsize=(8,5))
accax = 'Y'
plt.subplot(311)
plt.plot(bwindow*uteste[accax])
plt.title("Blackman Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')



plt.subplot(312)
plt.plot(hmwindow*uteste[accax])
plt.title("Hamming Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')


plt.subplot(313)
plt.plot(hnwindow*uteste[accax])
plt.title("Hanning Window")
plt.tick_params(axis='x')
plt.tick_params(axis='y')

plt.tight_layout()

Como é visível no plot dos gráficos, não efetuar um detrend aos dados pode levar a um formato não desejado do sinal, pelo que no nosso estudo, em principio será aplicado um dentrend a cada sinal, de forma a centrá-lo no eixo y=0.

In [ ]:
patamar = 0.7
#plot_activity_dft(uteste, len(uteste), 0, 0, 'WALK', False, patamar,True,'b')
#plot_activity_dft(uteste, len(uteste), 0, 0, 'WALK', False, patamar,True,'hm')
#plot_activity_dft(uteste, len(uteste), 0, 0, 'WALK', False, patamar,True,'hn')
#plot_activity_dft(uteste, len(uteste), 0, 0, 'WALK', False, patamar,True,'k')


Pela verificação do produto do das janelas, com o segmento do sinal associado a atividades dinâmicas e ao mesmo sinal após um 'detrend', verifica-se que o sinal detrended obtém melhores resultados para uma melhor análise, pois fica mais simétrico em relação ao eixo central. 

In [ ]:
x, y, z = plot_activity_dft(walks_down_user2_2_detrended[0], len(walks_down_user2_2_detrended[0]), 1, 2, 'WALK DOWN', False, 0,'blackman',True)
x, y, z = plot_activity_dft(walks_down_user2_2_detrended[0], len(walks_down_user2_2_detrended[0]), 1, 2, 'WALK DOWN', False, 0, 'hamming', True)
x, y, z = plot_activity_dft(walks_down_user2_2_detrended[0], len(walks_down_user2_2_detrended[0]), 1, 2, 'WALK DOWN', False, 0, 'hann', True)
x, y, z = plot_activity_dft(walks_down_user2_2_detrended[0], len(walks_down_user2_2_detrended[0]), 1, 2, 'WALK DOWN', False, 0, 'rect', True)

## Calculo das DFT's das várias experiências dos utilizadores do nosso dataset

Para fazer o cálculo das DFT's é usada a função 'plot_activity_dft(activity, size, user, exp, title, zeros, threshold, window, plotit)'
* activity: fragmento para calcular a DFT
* size: len(activity)
* user: nº do user
* exp: nº da experiência
* title: nome da atividade
* zeros: ignorar os frequências abaixo da frequência máxima * threshold
* window: tipo de janela a usar
* plotit: mostar o gráfico ou não

#### <u>Nota</u>: Apenas esta a ser mostrado o cálculo das DFT's para a 1º atividade de cada categoria, i.e, apenas o 1º <i>walking</i> dos vários users, porque cada experiência tem 3-4 <i>walkings</i>

In [ ]:
hannx, y, z = plot_activity_dft(walks_down_user1_1_detrended[0], len(walks_down_user1_1_detrended[0]), 1, 1, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user1_2_detrended[0], len(walks_down_user1_2_detrended[0]), 1, 2, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user2_1_detrended[0], len(walks_down_user2_1_detrended[0]), 2, 1, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user2_2_detrended[0], len(walks_down_user2_2_detrended[0]), 2, 2, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user3_1_detrended[0], len(walks_down_user3_1_detrended[0]), 3, 1, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user3_2_detrended[0], len(walks_down_user3_2_detrended[0]), 3, 2, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user4_1_detrended[0], len(walks_down_user4_1_detrended[0]), 4, 1, 'WALK DOWN', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_down_user4_2_detrended[0], len(walks_down_user4_2_detrended[0]), 4, 2, 'WALK DOWN', True, 0.7, 'hann', False)

### DFT's da atividades WALK_UP de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(walks_up_user1_1_detrended[0], len(walks_up_user1_1_detrended[0]), 1, 1, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user1_2_detrended[0], len(walks_up_user1_2_detrended[0]), 1, 2, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user2_1_detrended[0], len(walks_up_user2_1_detrended[0]), 2, 1, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user2_2_detrended[0], len(walks_up_user2_2_detrended[0]), 2, 2, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user3_1_detrended[0], len(walks_up_user3_1_detrended[0]), 3, 1, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user3_2_detrended[0], len(walks_up_user3_2_detrended[0]), 3, 2, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user4_1_detrended[0], len(walks_up_user4_1_detrended[0]), 4, 1, 'WALK UP', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_up_user4_2_detrended[0], len(walks_up_user4_2_detrended[0]), 4, 2, 'WALK UP', True, 0.7, 'hann', False)

### DFT's da atividades WALK de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(walks_user1_1_detrended[0], len(walks_user1_1_detrended[0]), 1, 1, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user1_2_detrended[0], len(walks_user1_2_detrended[0]), 1, 2, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user2_1_detrended[0], len(walks_user2_1_detrended[0]), 2, 1, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user2_2_detrended[0], len(walks_user2_2_detrended[0]), 2, 2, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user3_1_detrended[0], len(walks_user3_1_detrended[0]), 3, 1, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user3_2_detrended[0], len(walks_user3_2_detrended[0]), 3, 2, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user4_1_detrended[0], len(walks_user4_1_detrended[0]), 4, 1, 'WALK', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(walks_user4_2_detrended[0], len(walks_user4_2_detrended[0]), 4, 2, 'WALK', True, 0.7, 'hann', False)

### DFT's da atividades STAND de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(stand_user1_1_detrended[0], len(stand_user1_1_detrended[0]), 1, 1, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user1_2_detrended[0], len(stand_user1_2_detrended[0]), 1, 2, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user2_1_detrended[0], len(stand_user2_1_detrended[0]), 2, 1, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user2_2_detrended[0], len(stand_user2_2_detrended[0]), 2, 2, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user3_1_detrended[0], len(stand_user3_1_detrended[0]), 3, 1, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user3_2_detrended[0], len(stand_user3_2_detrended[0]), 3, 2, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user4_1_detrended[0], len(stand_user4_1_detrended[0]), 4, 1, 'STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_user4_2_detrended[0], len(stand_user4_2_detrended[0]), 4, 2, 'STAND', True, 0.7, 'hann', False)

### DFT's da atividades SIT de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(sit_user1_1_detrended[0], len(sit_user1_1_detrended[0]), 1, 1, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user1_2_detrended[0], len(sit_user1_2_detrended[0]), 1, 2, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user2_1_detrended[0], len(sit_user2_1_detrended[0]), 2, 1, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user2_2_detrended[0], len(sit_user2_2_detrended[0]), 2, 2, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user3_1_detrended[0], len(sit_user3_1_detrended[0]), 3, 1, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user3_2_detrended[0], len(sit_user3_2_detrended[0]), 3, 2, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user4_1_detrended[0], len(sit_user4_1_detrended[0]), 4, 1, 'SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_user4_2_detrended[0], len(sit_user4_2_detrended[0]), 4, 2, 'SIT', True, 0.7, 'hann', False)

### DFT's da atividades LAY de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(lay_user1_1_detrended[1], len(lay_user1_1_detrended[1]), 1, 1, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user1_2_detrended[1], len(lay_user1_2_detrended[1]), 1, 2, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user2_1_detrended[1], len(lay_user2_1_detrended[1]), 2, 1, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user2_2_detrended[1], len(lay_user2_2_detrended[1]), 2, 2, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user3_1_detrended[1], len(lay_user3_1_detrended[1]), 3, 1, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user3_2_detrended[1], len(lay_user3_2_detrended[1]), 3, 2, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user4_1_detrended[1], len(lay_user4_1_detrended[1]), 4, 1, 'LAY', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lay_user4_2_detrended[1], len(lay_user4_2_detrended[1]), 4, 2, 'LAY', True, 0.7, 'hann', False)

### DFT's da atividades STAND_SIT de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(stand_sit_user1_1_detrended, len(stand_sit_user1_1_detrended), 1, 1, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user1_2_detrended, len(stand_sit_user1_2_detrended), 1, 2, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user2_1_detrended, len(stand_sit_user2_1_detrended), 2, 1, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user2_2_detrended, len(stand_sit_user2_2_detrended), 2, 2, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user3_1_detrended, len(stand_sit_user3_1_detrended), 3, 1, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user3_2_detrended, len(stand_sit_user3_2_detrended), 3, 2, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user4_1_detrended, len(stand_sit_user4_1_detrended), 4, 1, 'STAND_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_sit_user4_2_detrended, len(stand_sit_user4_2_detrended), 4, 2, 'STAND_SIT', True, 0.7, 'hann', False)


### DFT's da atividades SIT_STAND de todos as experiências

In [ ]:
hann, y, z = plot_activity_dft(sit_stand_user1_1_detrended, len(sit_stand_user1_1_detrended), 1, 1, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user1_2_detrended, len(sit_stand_user1_2_detrended), 1, 2, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user2_1_detrended, len(sit_stand_user2_1_detrended), 2, 1, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user2_2_detrended, len(sit_stand_user2_2_detrended), 2, 2, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user3_1_detrended, len(sit_stand_user3_1_detrended), 3, 1, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user3_2_detrended, len(sit_stand_user3_2_detrended), 3, 2, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user4_1_detrended, len(sit_stand_user4_1_detrended), 4, 1, 'SIT_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_stand_user4_2_detrended, len(sit_stand_user4_2_detrended), 4, 2, 'SIT_STAND', True, 0.7, 'hann', False)

### DFT's da atividades STAND_LIE de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(stand_lie_user1_1_detrended, len(stand_lie_user1_1_detrended), 1, 1, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user1_2_detrended, len(stand_lie_user1_2_detrended), 1, 2, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user2_1_detrended, len(stand_lie_user2_1_detrended), 2, 1, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user2_2_detrended, len(stand_lie_user2_2_detrended), 2, 2, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user3_1_detrended, len(stand_lie_user3_1_detrended), 3, 1, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user3_2_detrended, len(stand_lie_user3_2_detrended), 3, 2, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user4_1_detrended, len(stand_lie_user4_1_detrended), 4, 1, 'STAND_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(stand_lie_user4_2_detrended, len(stand_lie_user4_2_detrended), 4, 2, 'STAND_LIE', True, 0.7, 'hann', False)

### DFT's da atividades LIE_STAND de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(lie_stand_user1_1_detrended, len(lie_stand_user1_1_detrended), 1, 1, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user1_2_detrended, len(lie_stand_user1_2_detrended), 1, 2, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user2_1_detrended, len(lie_stand_user2_1_detrended), 2, 1, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user2_2_detrended, len(lie_stand_user2_2_detrended), 2, 2, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user3_1_detrended, len(lie_stand_user3_1_detrended), 3, 1, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user3_2_detrended, len(lie_stand_user3_2_detrended), 3, 2, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user4_1_detrended, len(lie_stand_user4_1_detrended), 4, 1, 'LIE_STAND', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_stand_user4_2_detrended, len(lie_stand_user4_2_detrended), 4, 2, 'LIE_STAND', True, 0.7, 'hann', False)

### DFT's da atividades LIE_SIT de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(lie_sit_user1_1_detrended, len(lie_sit_user1_1_detrended), 1, 1, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user1_2_detrended, len(lie_sit_user1_2_detrended), 1, 2, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user2_1_detrended, len(lie_sit_user2_1_detrended), 2, 1, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user2_2_detrended, len(lie_sit_user2_2_detrended), 2, 2, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user3_1_detrended, len(lie_sit_user3_1_detrended), 3, 1, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user3_2_detrended, len(lie_sit_user3_2_detrended), 3, 2, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user4_1_detrended, len(lie_sit_user4_1_detrended), 4, 1, 'LIE_SIT', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(lie_sit_user4_2_detrended, len(lie_sit_user4_2_detrended), 4, 2, 'LIE_SIT', True, 0.7, 'hann', False)

### DFT's da atividades SIT_LIE de todos as experiências

In [ ]:
x, y, z = plot_activity_dft(sit_lie_user1_1_detrended, len(sit_lie_user1_1_detrended), 1, 1, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user1_2_detrended, len(sit_lie_user1_2_detrended), 1, 2, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user2_1_detrended, len(sit_lie_user2_1_detrended), 2, 1, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user2_2_detrended, len(sit_lie_user2_2_detrended), 2, 2, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user3_1_detrended, len(sit_lie_user3_1_detrended), 3, 1, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user3_2_detrended, len(sit_lie_user3_2_detrended), 3, 2, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user4_1_detrended, len(sit_lie_user4_1_detrended), 4, 1, 'SIT_LIE', True, 0.7, 'hann', False)
x, y, z = plot_activity_dft(sit_lie_user4_2_detrended, len(sit_lie_user4_2_detrended), 4, 2, 'SIT_LIE', True, 0.7, 'hann', False)

## Frequências representativas de cada atividade
Para obter as frequências que representam cada atividade, é usada a função 'get_frequencies_from_activities()' que da print ao 'describe()' de um <i>DataFrame</i> do módulo <i>Pandas</i> que tem a informação de uma atividade de todos os <i>users</i>. O 'describe()' mostra o valor minimo presente, o maximo, a média, o desvio padrão, entre outros. Com estes dados podemos escolher um intervalo de frequências, em cada eixo, que representam uma atividade. 

### Walking

In [ ]:
d = get_frequencies_from_activities(walks_user, 0.8, 'hann')
walk_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Walking down

In [ ]:
d = get_frequencies_from_activities(walks_down_user, 0.8, 'hann')
walk_down_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Walking up

In [ ]:
d = get_frequencies_from_activities(walks_up_user, 0.8, 'hann')
walk_up_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Stand

In [ ]:
d = get_frequencies_from_activities(stand_user, 0.8, 'hann')
stand_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Sit

In [ ]:
d = get_frequencies_from_activities(sit_user, 0.8, 'hann')
sit_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Lay

In [ ]:
d = get_frequencies_from_activities(lay_user, 0.8, 'hann')
lay_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Stand to sit

In [ ]:
d = get_frequencies_from_activities(stand_sit_user, 0.8, 'hann')
stand_sit_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Sit to stand

In [ ]:
d = get_frequencies_from_activities(sit_stand_user, 0.8, 'hann')
sit_stand_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Stand to lie

In [ ]:
d = get_frequencies_from_activities(stand_lie_user, 0.8, 'hann')
stand_lie_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Lie to stand

In [ ]:
d = get_frequencies_from_activities(lie_stand_user, 0.8, 'hann')
lie_stand_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Lie to sit

In [ ]:
d = get_frequencies_from_activities(lie_sit_user, 0.8, 'hann')
lie_sit_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

### Sit to lie

In [ ]:
d = get_frequencies_from_activities(sit_lie_user, 0.8, 'hann')
sit_lie_user_freq = {'X':[d['X'].min(), d['X'].max()], 'Y': [d['Y'].min(), d['Y'].max()], 'Z':[d['Z'].min(), d['Z'].max()]}
d.describe()

## Calculo dos passos por minuto
Para estimar os passos por minuto, é necessário ter o valor do período, ou da frequência do sinal corresponde à atividade.
Visto que não temos a frequência exata do sinal, é necessário, também, estimar este parâmetro. Esta estimativa é calculada através de uma média aritmética das frequências mais significativas presentes em cada segmento da atividade em estudo.<br>
O eixo das acelerações 'X' pareceu-nos o mais indicado para o cálculo, pois é o que sofre mais alterações em cada passo, visto termos oscilações verticais aquando do mesmo, principalmente na subida e descida de escadas. O eixo 'Y', no entanto, também seria um bom candidato, visto termos uma rotação de tronco que acompanha o passo.<br>
O número de passos por minuto é calculado através da seguinte fórmula: <br>
$$
steps = \frac{60}{\frac{1}{freqs[X]}}
$$

$\frac{1}{freqs[X]}$ = período do sinal. Apenas é necesário fazer $\frac{60}{T}$ para obter os passos por minuto.


### Walking

In [ ]:
aux = []
for i in range(4):
    for j in range(2):
        d = get_frequencies_from_activities(walks_user[i+j], 0.8, 'hann')
        steps = 60 / (1/d['X'].mean())
        aux.append(steps)
        print(f'user{i+1}_{j+1} -> {steps:.2f}')

d = pd.DataFrame(np.array(aux), columns = ['Steps p/ min'])
d.describe().loc[['mean','std']]

### Walking up

In [ ]:
aux = []
for i in range(4):
    for j in range(2):
        d = get_frequencies_from_activities(walks_up_user[i+j], 0.8, 'blackman')
        steps = 60 / (1/d['X'].mean())
        aux.append(steps)
        print(f'user{i+1}_{j+1} -> {steps:.2f}')

d = pd.DataFrame(np.array(aux), columns = ['Steps p/ min'])
d.describe().loc[['mean','std']]

### Walking down

In [ ]:
aux = []
for i in range(4):
    for j in range(2):
        d = get_frequencies_from_activities(walks_down_user[i+j], 0.8, 'blackman')
        steps = 60 / (1/d['X'].mean())
        aux.append(steps)
        print(f'user{i+1}_{j+1} -> {steps:.2f}')

d = pd.DataFrame(np.array(aux), columns = ['Steps p/ min'])
d.describe().loc[['mean','std']]

### <i>Print</i> das frequências representativas de cada atividade, obtidas anteriormente

In [ ]:
print('walk')
print(walk_user_freq)
print('walk down')
print(walk_down_user_freq)
print('walk up')
print(walk_up_user_freq)
print('stand')
print(stand_user_freq)
print('sit')
print(sit_user_freq)
print('lay')
print(lay_user_freq)
print('stand to sit')
print(stand_sit_user_freq)
print('sit to stand')
print(sit_stand_user_freq)
print('stand to lie')
print(stand_lie_user_freq)
print('lie to stand')
print(lie_stand_user_freq)
print('lie to sit')
print(lie_sit_user_freq)
print('sit to lie')
print(sit_lie_user_freq)

### Juntar as freqências num dicionário para ser mais fácil de fazer a indentificação

In [ ]:
atividades = {}
atividades['walk'] = walk_user_freq
atividades['walk_down'] = walk_down_user_freq
atividades['walk_up'] = walk_up_user_freq
atividades['stand'] = stand_user_freq
atividades['lay'] = lay_user_freq
atividades['sit'] = sit_user_freq
atividades['stand_sit'] = stand_sit_user_freq
atividades['sit_stand'] = sit_stand_user_freq
atividades['stand_lie'] = stand_lie_user_freq
atividades['lie_stand'] = lie_stand_user_freq
atividades['lie_sit'] = lie_sit_user_freq
atividades['sit_lie'] = sit_lie_user_freq

In [ ]:
#walks_index = np.array(list(filter(lambda x: x[2]==1, intervalos)))
#walks_down_index = np.array(list(filter(lambda x: x[2]==3, intervalos)))
#walks_up_index = np.array(list(filter(lambda x: x[2]==3, intervalos)))

#sit_index = np.array(list(filter(lambda x: x[2]==4, intervalos)))
#stand_index = np.array(list(filter(lambda x: x[2]==5, intervalos)))
#lay_index = np.array(list(filter(lambda x: x[2]==6, intervalos)))

#stand_sit_index = np.array(list(filter(lambda x: x[2]==7, intervalos)))
#sit_stand_index = np.array(list(filter(lambda x: x[2]==8, intervalos)))
#sit_lie_index = np.array(list(filter(lambda x: x[2]==9, intervalos)))
#lie_sit_index = np.array(list(filter(lambda x: x[2]==10, intervalos)))
#stand_lie_index = np.array(list(filter(lambda x: x[2]==11, intervalos)))
#lie_stand_index = np.array(list(filter(lambda x: x[2]==12, intervalos)))

## Métodos de distinguir as atividades de transição, dinâmicas e estáticas
Pela análise da sinal tempo, bem como da sua Transformada de Fourier, foi possível verificar que as atividades estáticas, em casos normais (ou seja, sem ruído), são caracterizadas por terem uma amplitude baixa. Num caso perfeito em que o ruído não estivesse presente, seriam também caracterizados por uma grande magnitude na frequência 0.<br>
Nas atividades de Transição, é claro, do lado do tempo, que estas têm uma duração de um instante, não perduram no tempo, pelo que para a verificação da duração do sinal nos pareceu um bom método.
Para as atividades dinâmicas, a verificação, é feita maioritariamente a partir da frequência do segmento do sinal.

### 3.5 Verificação da atividade presente em cada Segmento para teste à Sensibilidade e especificidade dos parâmetros usados para distinguir atividades

colocar um array com os resultados que são suposto obter (pela ordem do ficheiro 'activity_labels.txt')


In [ ]:
real = []
for i in intervalos:
    real.append( convert_to_class(i) )

Mapear os resultados para formato binário, para cada classe, de modo a obter uma melhor matriz confusão.
Isto é. Conforme a classe de atividades que estamos a testar colocar 1 onde é essa atividade, 0 onde não é essa atividade.
Estes são os valores de 'teste' para cada atividade.

In [ ]:
dinamicos = list(map(lambda x: 1 if x==3 else 0, real))
estaticos = list(map(lambda x: 1 if x==1 else 0, real))
transicao = list(map(lambda x: 1 if x==2 else 0, real))

In [ ]:
predict = []
for i in intervalos:
    value = get_sample_predict(get_user(i[1], i[0]), 1, [i[-2], i[-1]])
    predict.append( value )

de igual modo, mapear a resposta obtida para formato binaria conforme a atividade a testar.

In [ ]:
dinamicos_predict = list(map(lambda x: 1 if x==3 else 0, predict))
estaticos_predict = list(map(lambda x: 1 if x==1 else 0, predict))
transicao_predict = list(map(lambda x: 1 if x==2 else 0, predict))

##### Formatação da matriz de confusão da livraria sklearn:
$
\begin{array}{c|c}
    TrueNegative & FalsePositive
    \\ \hline
    FalseNegative & TruePosivite
\end{array}
$

In [ ]:
a = confusion_matrix(dinamicos, dinamicos_predict)
b = confusion_matrix(estaticos, estaticos_predict)
c = confusion_matrix(transicao, transicao_predict)

In [ ]:
md("""
### Matriz de confusão para atividades dinâmicas
$
\\begin{array}{c|c}
    %d & %d
    \\\\ \\hline
    %d & %d
\\end{array}
$
### Matriz de confusão para atividades estáticas
$
\\begin{array}{c|c}
    %d & %d
    \\\\ \\hline
    %d & %d
\\end{array}
$
### Matriz de confusão para atividades transição
$
\\begin{array}{c|c}
    %d & %d
    \\\\ \\hline
    %d & %d
\\end{array}
$
"""%(a[0,0], a[0,1], a[1,0], a[1,1], b[0,0], b[0,1], b[1,0], b[1,1], c[0,0], c[0,1], c[1,0], c[1,1]))


In [ ]:
md("""
### Sensibilidade para deteção de atividades dinâmicas:
$
\\frac{%d}{%d+%d} = %.2f \\%%
$ 
### Especificidade para deteção de atividades dinâmicas:
$ 
\\frac{%d}{%d+%d} = %.2f \\%%
$
"""%(a[1,1], a[1,1], a[1,0], a[1,1]/(a[1,1]+a[1,0])*100, a[0,0], a[0,0], a[0,1], a[0,0] / (a[0,0]+a[0,1])*100
))

In [ ]:
md("""
### Sensibilidade para detecao de atividade estatica:
$
\\frac{%d}{%d+%d} = %.2f \\%%
$ 
### Especificidade para deteção de atividade estática:
$ 
\\frac{%d}{%d+%d} = %.2f \\%%
$
"""%(b[1,1], b[1,1], b[1,0], b[1,1]/(b[1,1]+b[1,0])*100, b[0,0], b[0,0], b[0,1], b[0,0] / (b[0,0]+b[0,1])*100
))

In [ ]:
md("""
### Sensibilidade para detecao de atividades de transição:
$
\\frac{%d}{%d+%d} = %.2f \\%%
$ 
### Especificidade para deteção de atividades de transição:
$ 
\\frac{%d}{%d+%d} = %.2f \\%%
$
"""%(c[1,1], c[1,1], c[1,0], c[1,1]/(c[1,1]+c[1,0])*100, c[0,0], c[0,0], c[0,1], c[0,0] / (c[0,0]+c[0,1])*100
))

## 4. STFT - <i> Short Time Fourrier Transform </i>

Como na STFT, as janelas vão ter sempre uma duração pequena, é impraticável a verificação da duração do tempo do segmento para verificar transições. Assim, atualizámos o parâmetro para verificação de uma atividade de transição. Faz se acompanhar o sinal de um filtro de ruído, média móvel. Esta apenas sofre variação aquando de uma alteração brusca nos valores, e é quando isto acontece que presumimos ter acontecido uma atividade de transição.<br>
Isto pode ser verificado no plot na célula abaixo.

In [ ]:
v = 300 # numero de pontos para calcular a média aritmética
step = 300 # tamanho da janela
user = user1_1
media_movel = []
for i in range(v,len(user)):
    media_movel.append( (user['Z'][i-v:i-1]/v).sum() )

plt.figure(figsize=(15,4))
axe = plt.subplot(111)
user.plot('Time (min)','Z', ax = axe)
axe.plot(user['Time (min)'][v:], media_movel)
plt.vlines([(v+i)/60/50 for i in range(0, len(user), step)], plt.ylim()[0], plt.ylim()[1], linestyle = ':', color='k')

In [ ]:
unow = user1_1
size = 400
half = size//2
times, freqs, cms, var = stft(unow, size, 300)
times = np.append([0], times)
freqs = np.append([0], freqs)
cms = np.append([0], cms)
var = np.append([0], var)

t1 = times[::2]
t2 = times[1::2]

# i = numero da janela. Apenas para visualizar uma janela em específico
i = 10
fig = plt.figure(figsize=(20,15))
plt.subplot(411)
plt.title('Frequências')
plt.plot(times, freqs, 'o', c = 'royalblue')
plt.axvspan(times[i], times[i+2], facecolor='g', alpha=0.5)
plt.vlines(t1, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='red', lw = 0.6)
plt.vlines(t2, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='blue', lw = 0.6)

plt.subplot(412)
plt.title('Amplitudes máximas')
plt.plot(times, cms, 'o', c = 'darkslategrey')
plt.axvspan(times[i], times[i+2], facecolor='g', alpha=0.5)
plt.vlines(t1, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='red', lw = 0.6)
plt.vlines(t2, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='blue', lw = 0.6)

plt.subplot(413)
plt.title('Variações na média móvel')
plt.plot(times, var, 'or')
plt.axvspan(times[i], times[i+2], facecolor='g', alpha=0.5)
plt.vlines(t1, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='red', lw = 0.6)
plt.vlines(t2, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='blue', lw = 0.6)

plt.subplot(414)
x_axis = np.arange(0, len(unow['Z']) * (1/50), (1/50)) / 60
plt.plot(x_axis, unow['Z'], 'k', lw = 0.7)
plt.vlines(t1, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='red', lw = 0.6)
plt.vlines(t2, plt.ylim()[0] , plt.ylim()[1] , linestyle='dashed', color='blue', lw = 0.6)

a = int(times[i]*60*50)
b = int(times[i+2]*60*50)
plt.plot(x_axis[a:b], unow['Z'][a:b], 'g', lw = 1)
plt.axvspan(times[i], times[i+2], facecolor='g', alpha=0.5)

fig.tight_layout()

Plot das previsões

In [ ]:
cores = ['blue', 'olive', 'maroon', 'black']
#arranjar as cores
cores_to_plot = []
for i in range(1, len(freqs)):
    if var[i] > 0.1:
        
        if freqs[i] > 1.5 or np.round(cms[i], 2) > 0.01:
            cores_to_plot.append(cores[0])

        elif np.round(cms[i], 2) <= 0.01:
            cores_to_plot.append(cores[1])
        
        elif 0.5<=freqs[i]<=2:
            cores_to_plot.append(cores[2])
            
        else:
            cores_to_plot.append(cores[-1])
        
    else:
        if np.round(cms[i], 2) <= 0.01:
            cores_to_plot.append(cores[1])
            
        elif 0.5<=freqs[i]<=2 and np.round(cms[i], 3) > 0.025:
            cores_to_plot.append(cores[2])
            
        else:
            cores_to_plot.append(cores[-1])

In [ ]:
from matplotlib.lines import Line2D
array = [Line2D([0], [0], color='blue', lw=4), Line2D([0], [0], color='maroon', lw=4), Line2D([0], [0], color='olive', lw=4), Line2D([0], [0], color='black', lw=4)]

plt.figure(figsize = (16,4))
j=0
for  i in range(0, len(unow) - size + 1, half):
    plt.plot(unow['Time (min)'][i:i+size], unow['Z'][i:i+size], c = cores_to_plot[j], lw=0.5)
    j+=1

plt.tight_layout()
plt.legend(array, ['Transição', 'Dinâmica', 'Estática', 'Não indentificado'])

Os resultados obtidos, apesar de não serem maus, poderiam ser melhores. Para isso seria necessária mais informação, para se conseguir generalizar melhor a população. A previsão de atividades com uso de apenas um eixo, também limita bastante a previsão.<br>
Apesar de termos apenas previsto classe de atividades, verificámos diferenças entre as próprias atividades dinâmicas, como por exemplo a atividade <i> Walk_Downstairs </i> têm amplitudes superiores nas acelerações recebidas no eixo X, enquanto que a atividade <i> Walk_Upstairs </i> tem uma frequência menor nas acelerações do eixo Z. <br>
Pelos dados avaliados, que não são muitos, pelo que é complicado uma generalização, mas também há uma semelhança nas atividades estáticas, que são os intervalos das acelerações recebidas nos eixos Y. Estas são as unicas que se mantêm similares entre todas as experiências<br><br>
